In [4]:
from slither import Slither

# Disable SSL certificate verification
# Create a Slither instance assuming the smart contract code is verified on Etherscan

sl = Slither('0xdac17f958d2ee523a2206206994597c13d831ec7')


In [160]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('contracts_found.db')

# SQL query
query = "SELECT * FROM contracts_found"

# Reading the SQL query into a DataFrame
contracts_found = pd.read_sql_query(query, connection)
print(len(contracts_found))


252


In [73]:
for i in range(len(contracts_found)):
    if contracts_found.loc[i, 'success'] ==1:
        continue
    if 'Unknown language: unknown' in str(contracts_found.loc[i, 'error']):
        continue
        
    import time
    time.sleep(3)
    try:
                
        contract_address = contracts_found['Contract'][i]
        print(contract_address)
        sl = Slither(contract_address)
        contracts_found.loc[i, 'success'] = 1
        print(1)
    except Exception as e:
        print(e)
        
        contracts_found.loc[i, 'success'] = 0
        contracts_found.loc[i, 'error'] = e

0xfe9E45b7085eEb64D95903d3972c3712Af510d35
1
0x9C9747FCC731f8B665BeC6f0467C88604d671Ac9
1


In [ ]:
verified_contracts = contracts_found[contracts_found['success'] ==1]
verified_contracts

In [85]:
contracts_found[contracts_found.error.astype(str) !='Unknown language: unknown'].success.value_counts()

success
0.0    18
1.0    15
Name: count, dtype: int64

In [33]:
compilation_unit = sl.compilation_units[0]
# Print all the contracts from the USDT address
print([str(c) for c in compilation_unit.contracts])
# Print the most derived contracts from the USDT address
print([str(c) for c in compilation_unit.contracts_derived])

['IERC20', 'Context', 'Ownable', 'SafeMath', 'BaseToken', 'StandardToken']
['SafeMath', 'StandardToken']


In [150]:
contracts_found[contracts_found['success'] ==1]['Contract'].values[3]

'0xb8D44feCbAC44eA6Ea25D5480e0a902548870ed1'

In [157]:
# Your GraphQL query
api_key = 'BQY9GqLL6xadbpKY3trQHUOtvWs6HXUB'
import requests
query = """
{
  ethereum(network: ethereum) {
        dexTrades(options: {limit: 10, asc: "timeInterval.minute"}, 
        protocol: {is: "Uniswap v2"}, 
        date: { after: "2023-07-17T00:00:00Z" }
        buyCurrency: {is: "0x625933f04eb04f33Ac489c952854165C7cc4A2fd"}, 
        sellCurrency: {is: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}) {
            timeInterval {
                minute(count: 5)
            }
            buyCurrency {
                symbol
                address
            }
            buyAmount
            sellCurrency {
                symbol
                address
            }
            sellAmount
            trades: count
            maximum_price: price(calculate: maximum)
            minimum_price: price(calculate: minimum)
            open_price: minimum(of: block, get: price)
            close_price: maximum(of: block, get: price)
        }
    }
}
"""

# Headers including your API key
headers = {
    'Content-Type': 'application/json',
    'X-API-KEY': api_key
}

# Make the POST request
response = requests.post(url, json={'query': query}, headers=headers)

# Check the response
if response.status_code == 200:
    try:
        print(response.json())
    except ValueError:
        print("Failed to decode JSON from response:", response.text)
else:
    print("Failed to retrieve data:", response.status_code, response.text)

{'data': {'ethereum': {'dexTrades': [{'timeInterval': {'minute': '2024-03-01 01:30:00'}, 'buyCurrency': {'symbol': 'KSH', 'address': '0x625933f04eb04f33ac489c952854165c7cc4a2fd'}, 'buyAmount': 18740000.0, 'sellCurrency': {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}, 'sellAmount': 4.6320893331763076, 'trades': 4, 'maximum_price': 3.730063280229647e-07, 'minimum_price': 2.2404979889129066e-07, 'open_price': '2.4477729989881927e-07', 'close_price': '2.2404979889129066e-07'}, {'timeInterval': {'minute': '2024-03-01 01:35:00'}, 'buyCurrency': {'symbol': 'KSH', 'address': '0x625933f04eb04f33ac489c952854165c7cc4a2fd'}, 'buyAmount': 10602499.999999998, 'sellCurrency': {'symbol': 'WETH', 'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'}, 'sellAmount': 2.4497534662918223, 'trades': 4, 'maximum_price': 3.3220040407193227e-07, 'minimum_price': 1.9833629905508958e-07, 'open_price': '2.2257646929567424e-07', 'close_price': '2.08474824684788e-07'}, {'timeInterval

In [10]:
from web3 import Web3

# Connect to an Ethereum node (this should be your node or a service like Infura)


def get_supply(contract_address):
    web3 = Web3(Web3.HTTPProvider('https://eth.llamarpc.com'))
    contract_abi = '''
    [
        {
            "constant":true,
            "inputs":[],
            "name":"totalSupply",
            "outputs":[{"name":"","type":"uint256"}],
            "payable":false,
            "stateMutability":"view",
            "type":"function"
        }
    ]
    '''

    # Create the contract instance
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)

    # Call the totalSupply function
    total_supply = contract.functions.totalSupply().call()

    # Convert to a human-readable format (assuming the token uses 18 decimal places)
    total_supply_readable = total_supply / 10**18

    print("Total Supply:", total_supply_readable)
    return total_supply_readable